In [24]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OrdinalEncoder
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import train_test_split,RandomizedSearchCV,GridSearchCV,HalvingGridSearchCV,HalvingRandomSearchCV
from sklearn.ensemble import RandomForestRegressor

In [25]:
train=pd.read_csv('ParisHousing.csv')

# train=train.drop(['id'],axis=1)
test=pd.read_csv('test.csv')
print(train.shape,test.shape)


(10000, 17) (15154, 17)


In [26]:
corr_matrix=train.corr()
corr_matrix['price'].sort_values(ascending=False)

price                1.000000
squareMeters         0.999999
numPrevOwners        0.016619
numberOfRooms        0.009591
cityPartRange        0.008813
hasStormProtector    0.007496
floors               0.001654
attic               -0.000600
hasGuestRoom        -0.000644
cityCode            -0.001539
hasStorageRoom      -0.003485
basement            -0.003967
hasPool             -0.005070
hasYard             -0.006119
made                -0.007210
isNewBuilt          -0.010643
garage              -0.017229
Name: price, dtype: float64

In [27]:
sel=['price','squareMeters','numberOfRooms','floors','made','hasStormProtector','isNewBuilt','hasPool','hasStorageRoom']    
 

In [28]:
# train=train.loc[:,sel]
train

,squareMeters,numberOfRooms,hasYard,hasPool,floors,cityCode,cityPartRange,numPrevOwners,made,isNewBuilt,hasStormProtector,basement,attic,garage,hasStorageRoom,hasGuestRoom,price
0,75523,3,0,1,63,9373,3,8,2005,0,1,4313,9005,956,0,7,7559081.5
1,80771,39,1,1,98,39381,8,6,2015,1,0,3653,2436,128,1,2,8085989.5
2,55712,58,0,1,19,34457,6,8,2021,0,0,2937,8852,135,1,9,5574642.1
3,32316,47,0,0,6,27939,10,4,2012,0,1,659,7141,359,0,3,3232561.2
4,70429,19,1,1,90,38045,3,7,1990,1,0,8435,2429,292,1,4,7055052.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,1726,89,0,1,5,73133,7,6,2009,0,1,9311,1698,218,0,4,176425.9
9996,44403,29,1,1,12,34606,9,4,1990,0,1,9061,1742,230,0,0,4448474.0
9997,83841,3,0,0,69,80933,10,10,2005,1,1,8304,7730,345,1,9,8390030.5
9998,59036,70,0,0,96,55856,1,3,2010,0,1,2590,6174,339,1,4,5905107.0


In [29]:
sel=['squareMeters','numberOfRooms','floors','made','hasStormProtector','isNewBuilt','hasPool','hasStorageRoom']    

In [30]:
test.drop(['id'],axis=1,inplace=True)
# test=test.loc[:,sel]

In [31]:
from sklearn.preprocessing import StandardScaler,MinMaxScaler,LabelEncoder
s=StandardScaler()
m=MinMaxScaler()
l=LabelEncoder()


In [32]:
corr_matrix=train.corr()
corr_matrix['price'].sort_values(ascending=False)

price                1.000000
squareMeters         0.999999
numPrevOwners        0.016619
numberOfRooms        0.009591
cityPartRange        0.008813
hasStormProtector    0.007496
floors               0.001654
attic               -0.000600
hasGuestRoom        -0.000644
cityCode            -0.001539
hasStorageRoom      -0.003485
basement            -0.003967
hasPool             -0.005070
hasYard             -0.006119
made                -0.007210
isNewBuilt          -0.010643
garage              -0.017229
Name: price, dtype: float64

In [33]:
def pre(df):
    df.drop(['cityCode','numPrevOwners'],axis=1,inplace=True)
    df['squareMeters'] =m.fit_transform(df['squareMeters'].values.reshape(-1,1))
    df['basement'] =m.fit_transform(df['basement'].values.reshape(-1,1)) 
    df['attic'] =m.fit_transform(df['attic'].values.reshape(-1,1))
    df['garage'] =m.fit_transform(df['garage'].values.reshape(-1,1))
    df['numberOfRooms'] =m.fit_transform(df['numberOfRooms'].values.reshape(-1,1))
    df['floors'] =m.fit_transform(df['floors'].values.reshape(-1,1))
    df['cityPartRange'] =l.fit_transform(df['cityPartRange'])
    df['hasGuestRoom'] =l.fit_transform(df['hasGuestRoom'])
    


    return df

In [34]:
train=pre(train)
test=pre(test)

In [35]:
train

,squareMeters,numberOfRooms,hasYard,hasPool,floors,cityPartRange,made,isNewBuilt,hasStormProtector,basement,attic,garage,hasStorageRoom,hasGuestRoom,price
0,0.755020,0.020202,0,1,0.626263,2,2005,0,1,0.4313,0.900490,0.951111,0,7,7559081.5
1,0.807547,0.383838,1,1,0.979798,7,2015,1,0,0.3653,0.243524,0.031111,1,2,8085989.5
2,0.556731,0.575758,0,1,0.181818,5,2021,0,0,0.2937,0.885189,0.038889,1,9,5574642.1
3,0.322560,0.464646,0,0,0.050505,9,2012,0,1,0.0659,0.714071,0.287778,0,3,3232561.2
4,0.704034,0.181818,1,1,0.898990,2,1990,1,0,0.8435,0.242824,0.213333,1,4,7055052.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,0.016385,0.888889,0,1,0.040404,6,2009,0,1,0.9311,0.169717,0.131111,0,4,176425.9
9996,0.443539,0.282828,1,1,0.111111,8,1990,0,1,0.9061,0.174117,0.144444,0,0,4448474.0
9997,0.838274,0.020202,0,0,0.686869,9,2005,1,1,0.8304,0.772977,0.272222,1,9,8390030.5
9998,0.590001,0.696970,0,0,0.959596,0,2010,0,1,0.2590,0.617362,0.265556,1,4,5905107.0


In [36]:
test

,squareMeters,numberOfRooms,hasYard,hasPool,floors,cityPartRange,made,isNewBuilt,hasStormProtector,basement,attic,garage,hasStorageRoom,hasGuestRoom
0,0.475443,0.888889,0,1,0.070707,4,1995,0,0,0.077263,0.212280,0.234879,0,8
1,0.620574,0.373737,0,0,0.868687,9,1994,1,1,0.051602,0.239684,0.388105,1,4
2,0.904933,0.747475,1,1,0.363636,8,1997,0,1,0.083655,0.069523,0.299395,0,2
3,0.162964,0.464646,1,1,0.080808,5,2019,1,1,0.007831,0.132247,0.114919,1,5
4,0.674882,0.070707,0,0,0.545455,2,2014,1,1,0.041648,0.207038,0.396169,1,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15149,0.861753,0.737374,1,1,0.727273,1,1993,1,0,0.041637,0.101028,0.662298,1,1
15150,0.458481,0.404040,1,1,0.636364,3,2008,0,0,0.008707,0.197929,0.975806,0,0
15151,0.735365,0.505051,0,0,0.404040,9,2004,0,1,0.092115,0.046089,0.773185,0,10
15152,0.775893,0.505051,0,1,0.323232,3,2019,0,0,0.107518,0.251856,0.128024,0,2


In [37]:
# from sklearn.preprocessing import FunctionTransformer
# transf=FunctionTransformer(np.log1p)
# train['price']=transf.fit_transform(train['price'].values.reshape(-1,1))
# train

In [38]:
from sklearn.model_selection import train_test_split
X=train.drop(['price'],axis=1)
y=train['price']
# y=scaler.fit_transform(y.values.reshape(-1,1))



In [39]:
from sklearn.feature_selection import SelectKBest,mutual_info_regression

sel=SelectKBest(mutual_info_regression,k=7)
sel.fit(X,y)
xx=sel.get_support()
x=X.loc[:,xx]

x

,squareMeters,numberOfRooms,hasYard,floors,cityPartRange,isNewBuilt,attic
0,0.755020,0.020202,0,0.626263,2,0,0.900490
1,0.807547,0.383838,1,0.979798,7,1,0.243524
2,0.556731,0.575758,0,0.181818,5,0,0.885189
3,0.322560,0.464646,0,0.050505,9,0,0.714071
4,0.704034,0.181818,1,0.898990,2,1,0.242824
...,...,...,...,...,...,...,...
9995,0.016385,0.888889,0,0.040404,6,0,0.169717
9996,0.443539,0.282828,1,0.111111,8,0,0.174117
9997,0.838274,0.020202,0,0.686869,9,1,0.772977
9998,0.590001,0.696970,0,0.959596,0,0,0.617362


In [40]:
test=test.loc[:,xx]
test

,squareMeters,numberOfRooms,hasYard,floors,cityPartRange,isNewBuilt,attic
0,0.475443,0.888889,0,0.070707,4,0,0.212280
1,0.620574,0.373737,0,0.868687,9,1,0.239684
2,0.904933,0.747475,1,0.363636,8,0,0.069523
3,0.162964,0.464646,1,0.080808,5,1,0.132247
4,0.674882,0.070707,0,0.545455,2,1,0.207038
...,...,...,...,...,...,...,...
15149,0.861753,0.737374,1,0.727273,1,1,0.101028
15150,0.458481,0.404040,1,0.636364,3,0,0.197929
15151,0.735365,0.505051,0,0.404040,9,0,0.046089
15152,0.775893,0.505051,0,0.323232,3,0,0.251856


In [41]:
# from sklearn.preprocessing import PolynomialFeatures
# poly=PolynomialFeatures(degree=2,include_bias=False)
# x=poly.fit_transform(x)
# test=poly.fit_transform(test)
# x=pd.DataFrame(x)
# test=pd.DataFrame(test)

In [42]:
x

,squareMeters,numberOfRooms,hasYard,floors,cityPartRange,isNewBuilt,attic
0,0.755020,0.020202,0,0.626263,2,0,0.900490
1,0.807547,0.383838,1,0.979798,7,1,0.243524
2,0.556731,0.575758,0,0.181818,5,0,0.885189
3,0.322560,0.464646,0,0.050505,9,0,0.714071
4,0.704034,0.181818,1,0.898990,2,1,0.242824
...,...,...,...,...,...,...,...
9995,0.016385,0.888889,0,0.040404,6,0,0.169717
9996,0.443539,0.282828,1,0.111111,8,0,0.174117
9997,0.838274,0.020202,0,0.686869,9,1,0.772977
9998,0.590001,0.696970,0,0.959596,0,0,0.617362


In [43]:
test

,squareMeters,numberOfRooms,hasYard,floors,cityPartRange,isNewBuilt,attic
0,0.475443,0.888889,0,0.070707,4,0,0.212280
1,0.620574,0.373737,0,0.868687,9,1,0.239684
2,0.904933,0.747475,1,0.363636,8,0,0.069523
3,0.162964,0.464646,1,0.080808,5,1,0.132247
4,0.674882,0.070707,0,0.545455,2,1,0.207038
...,...,...,...,...,...,...,...
15149,0.861753,0.737374,1,0.727273,1,1,0.101028
15150,0.458481,0.404040,1,0.636364,3,0,0.197929
15151,0.735365,0.505051,0,0.404040,9,0,0.046089
15152,0.775893,0.505051,0,0.323232,3,0,0.251856


In [44]:
X_train,X_test,y_train,y_test=train_test_split(x.values,y,test_size=0.2,random_state=42)

In [45]:
from sklearn.linear_model import LinearRegression

from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

lr=LinearRegression()
lr.fit(X_train,y_train)

y_pred=lr.predict(X_test)
# y_pred=scaler.inverse_transform(y_pred)
print('RMSE:',np.sqrt(mean_squared_error(y_test,y_pred)))
print('R2:',r2_score(y_test,y_pred))
accuracy=lr.score(X_test,y_test)
print(accuracy)



RMSE: 2415.198357904746
R2: 0.9999993340169447
0.9999993340169447


In [46]:
# #do random forest regression
# from sklearn.ensemble import RandomForestRegressor
# p={'n_estimators': 1550,
#  'min_samples_split': 2,
#  'min_samples_leaf': 1,
#  'max_features': 'auto',
#  'max_depth': 85,
#  'bootstrap': True}
# rf=RandomForestRegressor(**p)
# rf.fit(X_train,y_train)
# y_pred=rf.predict(X_test)
# print('RMSE:',np.sqrt(mean_squared_error(y_test,y_pred)))
# print('R2:',r2_score(y_test,y_pred))
# accuracy=rf.score(X_test,y_test)
# print(accuracy)

In [47]:
# random_accuracy = Evaluation(rf,X_test,y_test)
# random_accuracy

In [48]:
p={'n_estimators': 127, 'num_leaves': 49, 'min_child_samples': 2, 'learning_rate': 0.0868816857468006, 'log_max_bin': 10, 'colsample_bytree': 1.0, 'reg_alpha': 0.010507224412752626, 'reg_lambda': 10.701330730544898}
#do lightgbm regression
import lightgbm as lgb
lgbm=lgb.LGBMRegressor()
lgbm.fit(X_train,y_train)
y_pred=lgbm.predict(X_test)
print('RMSE:',np.sqrt(mean_squared_error(y_test,y_pred)))
print('R2:',r2_score(y_test,y_pred))
accuracy=lgbm.score(X_test,y_test)
print(accuracy)


RMSE: 12567.090859246962
R2: 0.9999819686983004
0.9999819686983004


In [49]:
from HROCH import PHCRegressor
phc=PHCRegressor()
phc.fit(X_train,y_train)
y_pred=phc.predict(X_test)

print('RMSE:',np.sqrt(mean_squared_error(y_test,y_pred)))
print('R2:',r2_score(y_test,y_pred))
accuracy=phc.score(X_test,y_test)
print(accuracy)

RMSE: 2609.5563861224937
R2: 0.9999992225169303
0.9999992225169303


In [50]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split

# Load and preprocess data


# Reshape data for CNN
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

# Create CNN model
model = keras.Sequential([
    keras.layers.Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], 1)),
    keras.layers.MaxPooling1D(pool_size=2),
    keras.layers.Flatten(),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(1)
])

# Compile and train model
model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))


Epoch 1/10
250/250 [==============================] - 1s 1ms/step - loss: 32794606567424.0000 - val_loss: 34871011442688.0000
Epoch 2/10
250/250 [==============================] - 0s 969us/step - loss: 32762679525376.0000 - val_loss: 34803694960640.0000
Epoch 3/10
250/250 [==============================] - 0s 1ms/step - loss: 32646436487168.0000 - val_loss: 34623551700992.0000
Epoch 4/10
250/250 [==============================] - 0s 1ms/step - loss: 32399580725248.0000 - val_loss: 34291935346688.0000
Epoch 5/10
250/250 [==============================] - 0s 958us/step - loss: 31992590630912.0000 - val_loss: 33785183731712.0000
Epoch 6/10
250/250 [==============================] - 0s 1ms/step - loss: 31405316767744.0000 - val_loss: 33086521737216.0000
Epoch 7/10
250/250 [==============================] - 0s 973us/step - loss: 30631973093376.0000 - val_loss: 32196494622720.0000
Epoch 8/10
250/250 [==============================] - 0s 947us/step - loss: 29676013617152.0000 - val_loss: 3111

In [52]:
model.evaluate(X_test, y_test)

63/63 [==============================] - 0s 1ms/step - loss: 28477342351360.0000


28477342351360.0

In [51]:
y_pred = model.predict(test1)

y_pred

NameError: name 'test1' is not defined

In [ ]:
# from catboost import CatBoostRegressor
# cat=CatBoostRegressor(iterations=1000,learning_rate=0.1,depth=10,loss_function='RMSE')
# cat.fit(X_train,y_train)
# y_pred=cat.predict(X_test)
# print('RMSE:',np.sqrt(mean_squared_error(y_test,y_pred)))
# print('R2:',r2_score(y_test,y_pred))
# accuracy=cat.score(X_test,y_test)
# print(accuracy)

In [ ]:
# from xgboost import XGBRegressor
# model = XGBRegressor(base_score=0.5, n_estimators=5000, n_jobs=1,
#                      booster='gbtree', max_depth=4, min_child_weight=1, 
#                      colsample_bylevel=1,colsample_bynode=1, nthread=3,
#                      colsample_bytree=0.8, learning_rate=0.05,
#                      importance_type='gain', subsample=0.8,
#                      reg_alpha=1, reg_lambda=1, scale_pos_weight=1,
#                      objective="reg:squarederror", 
#                      random_state=42,seed=123, 
#                      silent=None, verbosity=1)
# model.fit(X_train, y_train)
# y_pred=model.predict(X_test)
# print('RMSE:',np.sqrt(mean_squared_error(y_test,y_pred)))
# print('R2:',r2_score(y_test,y_pred))


In [ ]:
# lgbm=lgb.LGBMRegressor(**p)
# lgbm.fit(X,y)

In [ ]:
c=pd.read_csv('sample_submission.csv')
c.drop(['price'],axis=1,inplace=True)
c

,id
0,22730
1,22731
2,22732
3,22733
4,22734
...,...
15149,37879
15150,37880
15151,37881
15152,37882


In [ ]:

x1=lgbm.predict(test)
# x1=transf.inverse_transform(x1)
c['price']=y_pred
c

,id,price
0,22730,4749318.00
1,22731,6193482.00
2,22732,9035294.00
3,22733,1629346.75
4,22734,6745179.00
...,...,...
15149,37879,8614390.00
15150,37880,4586243.50
15151,37881,7339300.50
15152,37882,7752058.50


In [ ]:
c.to_csv('phc1.csv',index=False)

In [ ]:
d=pd.read_csv('phc.csv')

In [ ]:
from sklearn.metrics import mean_squared_error

r=mean_squared_error(c['price'],d['price'])
np.sqrt(r)

78829.15390112397